In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import HTML

In [2]:
import bqplot as bq
from bqplot import Figure, Scatter, Axis, LinearScale

In [3]:
import copy
import traitlets as tr

map_colors = {'Heating': '#c22324',
 'Domestic Hot Water': '#ff7f0e',
 'Heating & Domestic Hot Water': '#eb4f18',
 'Cooling': '#1f77b4',
 'Auxiliary': '#9467bd',
 'Lighting': '#17becf',
 'Small Power': '#d3d3d3',
 'IT/Servers': '#8c564b',
 'PV Generation': '#bcbd22',
 'Catering': '#2ca02c',
 'Lifts': '#e377c2',
 'Other': '#000000',
 'Unknown': '#7C7270'}

labels = ['Heating', 'Domestic Hot Water', 'Cooling', 'Auxiliary', 'Lighting', 'Small Power', 'PV Generation']
colors = [map_colors[x] for x in labels]

class EnergyConsumption(widgets.VBox):     
    def __init__(self, data: dict[str, np.array]):
        if "existing building" not in data:
            raise ValueError("`existing building` data missing.")
        self.data = data
        options = [k for k in data.keys() if k != "existing building"]
        self.select = widgets.SelectMultiple(options=options, layout={"height": "120px"})
        self.xs = bq.OrdinalScale()  # ordinal scale to represent categorical data
        self.ys = bq.LinearScale()
        self.xax = bq.Axis(scale=self.xs, grid_lines="none")  # no grid lines needed for x
        self.yax = bq.Axis(
            scale=self.ys, orientation="vertical", label="Energy Consumption (kWh/m2/yr)"
        )
        self.fig = bq.Figure(title="Energy Consumption", axes=[self.xax, self.yax])
        super().__init__([self.select, self.fig])
        self._init_controls()
        self._set_chart()

    def _init_controls(self):
        self.select.observe(self._set_chart)
    
    def _get_y(self, data: dict[str, np.array]) -> np.array:
        data_copy = copy.deepcopy(data)
        baseline = data_copy.pop("existing building")
        arrays = np.array(list(data_copy.values()))
        input_y = np.array([baseline])
        for idx, array in enumerate(arrays):
            diff = input_y[-1] - arrays[idx]
            input_y = np.vstack([input_y, diff])
        return input_y

    def _set_chart(self, onchange=None):
        filtered_data = {"existing building": self.data["existing building"]} | {k: v for k, v in self.data.items() if k in self.select.value}
        self.bar = bq.Bars(
            x=np.array(list(filtered_data.keys())),
            y=np.transpose(self._get_y(filtered_data)),
            scales={"x": self.xs, "y": self.ys},
            type="stacked",
            colors=colors,
            labels=labels,
            display_legend=True,
            padding=0.4,
        )
        self.xs.domain = ["existing building"] + list(self.select.value)
        self.fig.marks = [self.bar]
        

data = {
    "existing building": np.array([25, 23, 11, 13, 17, 15, 0]),
    "replace glazing": np.array([8, 3, 2, 1, 1, 3, 0]),
    "improve airtightness": np.array([4, 3, 2, 1, 1, 3, 0]),
    "replace heat source with an efficient ASHP": np.array([4, 3, 2, 1, 1, 3, 0]),
    "update lighting to LED": np.array([3, 3, 2, 1, 1, 3, 0]),
    "add PV panels": np.array([2, 3, 1, 1, 1, 3, 0]),
}

chart = EnergyConsumption(data)
display(chart)

EnergyConsumption(children=(SelectMultiple(layout=Layout(height='120px'), options=('replace glazing', 'improve…